In [3]:
import pulp
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import itertools
import copy

In d:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In d:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In d:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [4]:
# Working through the hooks puzzles.
# Solving the matrix is fast using the same tools as the Feb 14 puzzle 
# the checks are too slow to loop through all posibilities of the hooks 
# so look to solve hook by hook 

### Puzzle details
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2018/02/hook3_puzzle-01.png" width="500" height="600">

In [26]:
#Setup the constraints
top_labels = [0,35,42,18,18.0, 36, 63,0]
bot_labels = [0,40,32,40,10,12,0,56,0]
left_labels = [0,56,0,32,40,15,16,25,0]
right_labels =[0,49,63,0,18,42,63,54,0]
params = [top_labels,bot_labels,left_labels,right_labels]

In [61]:
  
    def possible(self,row,col,digit):
        self.grid[row,col] = digit
        #check counts
        if np.sum((self.y*self.grid) == y[row,col]) > y[row,col]:
            self.grid[row,col] = -1 
            print(self.grid)
            return False

        #check looks
        #if self.check_looks(row,col,digit):
        #    self.grid[row,col] = -1  
        #    return False
        
        self.grid[row,col] = -1                          
        return True
    

    def check_looks(self,row,col,digit):
        if self.left_looks[row] > 0:
            found = 0
            total = 1
            for i in range(9):
                if found < 2:
                    if self.grid[row,i] != 0:
                        found +=1
                        total = total * (self.y*self.grid)[row,i]
                if found == 2:
                    if total != self.left_looks[row]:
                        return True
                       
    
        if self.top_looks[col] > 0:
            found = 0
            total = 1
            for i in range(9):
                if found < 2:
                    if self.grid[i,col] != 0:
                        found +=1
                        total = total * (self.y*self.grid)[i,col]
                if found == 2:
                    if total != self.top_looks[col]:
                        return True

        if self.right_looks[row] > 0:
            found = 0
            total = 1
            for i in range(8,-1,-1):
                if found < 2:
                    if self.grid[row,i] != 0:
                        found +=1
                        total = total * (self.y*self.grid)[row,i]
                if found == 2:
                    if total != self.right_looks[row]:
                        return True
    
        if self.bot_looks[col] > 0:
            found = 0
            total = 1
            for i in range(8,-1,-1):
                if found < 2:
                    if self.grid[i,col] != 0:
                        found +=1
                        total = total * (self.y*self.grid)[i,col]
                if found == 2:
                    if total != self.top_looks[col]:
                        return True
        return False    


In [ ]:
class Matrix():
    def __init__(self,top_labels,bot_labels,left_labels,right_labels):
        self.valid_grids= [np.ones((9,9),dtype=int)*-1
        self.coords = [0,0,9,9]
        self.alignment = [0,0,0,0,0,0,0,0,0]
        self.lvl = 9
        self.top_looks = top_labels
        self.bot_looks = bot_labels
        self.left_looks = left_labels
        self.right_looks = right_labels
    
    
    def add_layer(self,lvl,alignment):
        row_start,col_start,row_end, col_end = self.coords
    
        if alignment == 0:
            self.grid[row_start:row_end,col_start] =lvl
            self.grid[row_start,col_start:col_end] =lvl
            row_start +=1
            col_start +=1
    
        if alignment == 1:
            self.grid[row_start:row_end,col_start] =lvl
            self.grid[row_end-1,col_start:col_end] =lvl
            row_end -=1
            col_start +=1 

        if alignment == 2:
            self.grid[row_start:row_end,col_end-1] =lvl
            self.grid[row_start,col_start:col_end] =lvl
            row_start +=1
            col_end -=1
        
        if alignment == 3:
            self.grid[row_start:row_end,col_end-1] =lvl
            self.grid[row_end-1,col_start:col_end] =lvl
            row_end -=1
            col_end -=1 
    
        self.coords = [row_start,col_start,row_end, col_end]



### Puzzle solution
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2018/03/20180228_hooks_3_ans.png" width="500" height="600">

In [40]:
def add_layer(grid,coords,lvl,alignment):
        row_start,col_start,row_end, col_end = coords
    
        if alignment == 0:
            grid[row_start:row_end,col_start] =lvl
            grid[row_start,col_start:col_end] =lvl
            row_start +=1
            col_start +=1
    
        if alignment == 1:
            grid[row_start:row_end,col_start] =lvl
            grid[row_end-1,col_start:col_end] =lvl
            row_end -=1
            col_start +=1 

        if alignment == 2:
            grid[row_start:row_end,col_end-1] =lvl
            grid[row_start,col_start:col_end] =lvl
            row_start +=1
            col_end -=1
        
        if alignment == 3:
            grid[row_start:row_end,col_end-1] =lvl
            grid[row_end-1,col_start:col_end] =lvl
            row_end -=1
            col_end -=1 
    
        coords = [row_start,col_start,row_end, col_end]
        return grid,coords
    
def solve_layer(grid,lvl,coords,params):
    row_start,col_start,row_end, col_end = coords
    top_labels,bot_labels,left_labels,right_labels =params
    nums = lvl*2-1
    
    problem = pulp.LpProblem('Problem') 
    mask = pulp.LpVariable.dicts('x', [ num for num in range(nums)],lowBound=0,upBound=1, cat='Integer') # declare decision variables

    problem += pulp.lpSum([mask[num]for num in range(nums) ]) == lvl
    

In [ ]:
lvl = 9
grid = np.ones((9,9),dtype=int)*-1
coords = [0,0,9,9]

for i in range(4):
    temp_grid = copy.copy(grid)
    temp_coords = copy.copy(coords)
    grid,coords=add_layer(grid,coords,lvl,i)
    mask=solve_layer(grid,lvl,temp_coords,params)
    print(grid)
    grid = copy.copy(temp_grid)
    coords = copy.copy(temp_coords)
    
    

In [44]:
grid = np.ones((9,9),dtype=int)*-1

lvl = np.ones(17)*9

for row,col in itertools.product(range(row_start:row_end,col



In [45]:
grid = np.ones((9,9),dtype=int)*-1
coords = [0,0,9,9]
alignment = [0,0,0,0,0,0,0,0,0]

for i in range(9,0,-1):
    grid,coords=add_layer(grid,coords,i,alignment[i-1])
    
print(grid)
    

[[9 9 9 9 9 9 9 9 9]
 [9 8 8 8 8 8 8 8 8]
 [9 8 7 7 7 7 7 7 7]
 [9 8 7 6 6 6 6 6 6]
 [9 8 7 6 5 5 5 5 5]
 [9 8 7 6 5 4 4 4 4]
 [9 8 7 6 5 4 3 3 3]
 [9 8 7 6 5 4 3 2 2]
 [9 8 7 6 5 4 3 2 1]]
